> The goal is to create a model that is stable and performs well in the future.

#### Load the data

In [4]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# dataPath = "../home-credit-credit-risk-model-stability/"
dataPath = "../data/"
parquetPath = dataPath + "parquet_files/"
parquetTrain = parquetPath + "train/"
parquetTest = parquetPath + "test/"

In [5]:
# Following function is just an example

def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement all desired dtypes for tables
    for col in df.columns:
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [6]:
train_base = pl.read_parquet(parquetTrain + 'train_base.parquet')
train_static = pl.concat(
    [
        pl.read_parquet(parquetTrain + 'train_static_0_0.parquet').pipe(set_table_dtypes),
        pl.read_parquet(parquetTrain + 'train_static_0_1.parquet').pipe(set_table_dtypes),        
    ],
    how='vertical_relaxed',
)

train_static_cb = pl.read_parquet(parquetTrain + 'train_static_cb_0.parquet').pipe(set_table_dtypes)
train_person_1 = pl.read_parquet(parquetTrain + 'train_person_1.parquet').pipe(set_table_dtypes)
train_credit_bureau_b_2 = pl.read_parquet(parquetTrain + 'train_credit_bureau_b_2.parquet').pipe(set_table_dtypes)

## add line
train_deposit_1 = pl.read_parquet(parquetTrain + 'train_deposit_1.parquet').pipe(set_table_dtypes)


In [7]:
test_basetable = pl.read_parquet(dataPath + "parquet_files/test/test_base.parquet")
test_static = pl.concat(
    [
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_0.parquet").pipe(set_table_dtypes),
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_1.parquet").pipe(set_table_dtypes),
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_2.parquet").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_parquet(dataPath + "parquet_files/test/test_static_cb_0.parquet").pipe(set_table_dtypes)
test_person_1 = pl.read_parquet(dataPath + "parquet_files/test/test_person_1.parquet").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_parquet(dataPath + "parquet_files/test/test_credit_bureau_b_2.parquet").pipe(set_table_dtypes) 

## add line
test_deposit_1 = pl.read_parquet(dataPath + 'parquet_files/test/test_deposit_1.parquet').pipe(set_table_dtypes)

#### Feature engineering

In [8]:
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

In [9]:
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

In [10]:
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by('case_id').agg(
    pl.col('pmts_pmtsoverdue_635A').max().alias('pmts_pmtsoverdue_635A_max'),
    (pl.col('pmts_dpdvalue_108P') > 31).max().alias('pmts_dpdvalue_108P_over31')
)

In [18]:
train_deposit_1_feats = train_deposit_1.group_by('case_id').agg(pl.sum('amount_416A').alias('total_amount_A')).sort(by='case_id')

In [19]:
train_deposit_1_feats.head()

case_id,total_amount_A
i64,f64
225,0.0
331,260.374
358,0.0
390,212175.81201
445,23735.938


In [20]:
# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [21]:
# Join all tables together.
data = train_base.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
).join( ## add line
    train_deposit_1_feats, how= "left", on="case_id"
)

In [22]:
## Same on test
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

## add line
test_deposit_1_feats = test_deposit_1.group_by('case_id').agg(pl.sum('amount_416A').alias('total_amount_A')).sort(by='case_id')
####

data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
). join( # add
    test_deposit_1_feats, how="left", on="case_id"
)

In [23]:
case_ids = data['case_id'].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [24]:
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

In [25]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

Train: (915995, 49)
Valid: (305332, 49)
Test: (305332, 49)


#### Training LightGBM

In [26]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

c:\Users\hmk40\anaconda3\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.706534
[100]	valid_0's auc: 0.725415
[150]	valid_0's auc: 0.732212
[200]	valid_0's auc: 0.736712
[250]	valid_0's auc: 0.739918
[300]	valid_0's auc: 0.741934
[350]	valid_0's auc: 0.743881
[400]	valid_0's auc: 0.745477
[450]	valid_0's auc: 0.746852
[500]	valid_0's auc: 0.747866
[550]	valid_0's auc: 0.748753
[600]	valid_0's auc: 0.749831
[650]	valid_0's auc: 0.750841
[700]	valid_0's auc: 0.751625
[750]	valid_0's auc: 0.752173
Early stopping, best iteration is:
[784]	valid_0's auc: 0.752741


In [27]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}') 

The AUC score on the train set is: 0.7658777233931774
The AUC score on the valid set is: 0.7527407728341027
The AUC score on the test set is: 0.7488949126642412


In [28]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}') 

The stability score on the train set is: 0.5011252123586621
The stability score on the valid set is: 0.46959297053655136
The stability score on the test set is: 0.45844926348477866


#### Submission

In [29]:
X_submission = data_submission[cols_pred].to_pandas()
X_submission = convert_strings(X_submission)
categorical_cols = X_train.select_dtypes(include=['category']).columns

for col in categorical_cols:
    train_categories = set(X_train[col].cat.categories)
    submission_categories = set(X_submission[col].cat.categories)
    new_categories = submission_categories - train_categories
    X_submission.loc[X_submission[col].isin(new_categories), col] = "Unknown"
    new_dtype = pd.CategoricalDtype(categories=train_categories, ordered=True)
    X_train[col] = X_train[col].astype(new_dtype)
    X_submission[col] = X_submission[col].astype(new_dtype)

y_submission_pred = gbm.predict(X_submission, num_iteration=gbm.best_iteration)

In [31]:
submission = pd.DataFrame({
    "case_id": data_submission["case_id"].to_numpy(),
    "score": y_submission_pred
}).set_index('case_id')
submission.to_csv("./submission_deposit.csv")